# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
#change if needed
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sys

#machine learning
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report, fbeta_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats.mstats import gmean

import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

%matplotlib inline

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table(table_name='message_categories', con=engine)
X = df['message']
Y = df.iloc[:,4:]

In [3]:
df.isna().sum()

id                            0
message                       0
original                  15990
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_infrastructure          0
weather_

In [4]:
display(X, Y)

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26023    The training demonstrated how to enhance micro...
26024    A suitable candidate has been selected and OCH...
26025    Proshika, operating in Cox's Bazar municipalit...
26026    Some 2,000 women protesting against the conduc...
26027    A radical shift in thinking came about as a re...
Name: message, Length: 26028, dtype: object

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26023,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26024,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26025,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26026,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [ ]:
#nlp
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


In [6]:
def tokenize(text):
    '''
    IN: 
        raw text for tokenizing via the following steps: 
            - normalized, punctuation removed, stop words removed, stemmed, and lemmatized
    OUT:
        tokenized text
    '''
    #Normalize text and remove punctuation
    normalized_txt = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize text
    words = word_tokenize(normalized_txt)
 
    #remove stop words
    #words = [w for w in words if w not in stopwords.words("english")]
    
    #Reduce words to their stems
    words = [PorterStemmer().stem(w) for w in words]
    
    #lemmatize
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
#starting pipeline
def pipeline_model():
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                          ('tfidf', TfidfTransformer()),
                          ('clf', MultiOutputClassifier(RandomForestClassifier()))])
    return pipeline

In [9]:
model = pipeline_model()

In [10]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [ ]:
# NEW PIPELINE (AFTER SOME TRIAL-AND-ERROR ALGORITHM TEST AND GRID SEARCH)
def new_model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,random_state = 42)

In [12]:
%%time
model.fit(X_train, Y_train)

Wall time: 4min 57s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
%%time
#make predictions
y_preds = model.predict(X_test)

Wall time: 16.2 s


In [17]:
results_dict = {}

for pred, label, col in zip(y_preds.transpose(), Y_test.values.transpose(), Y_test.columns):
    print(col)
    print(classification_report(label, pred))
    results_dict[col] = classification_report(label, pred, output_dict=True)

related
              precision    recall  f1-score   support

           0       0.76      0.29      0.42      1569
           1       0.81      0.97      0.88      4938

    accuracy                           0.81      6507
   macro avg       0.79      0.63      0.65      6507
weighted avg       0.80      0.81      0.77      6507

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5403
           1       0.87      0.46      0.60      1104

    accuracy                           0.90      6507
   macro avg       0.88      0.72      0.77      6507
weighted avg       0.89      0.90      0.88      6507

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5909
           1       0.85      0.31      0.46       598

    accuracy                           0.93      6507
   macro avg       0.89      0.65      0.71      6507
weighted avg       0.93      0.93      0.92      6507

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6406
           1       0.83      0.10      0.18       101

    accuracy                           0.99      6507
   macro avg       0.91      0.55      0.58      6507
weighted avg       0.98      0.99      0.98      6507

money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6339
           1       0.78      0.04      0.08       168

    accuracy                           0.97      6507
   macro avg       0.88      0.52      0.53      6507
weighted avg       0.97      0.97      0.96      6507

mis

In [20]:
results_dict

{'related': {'0': {'precision': 0.7601351351351351,
   'recall': 0.28680688336520077,
   'f1-score': 0.4164738546968995,
   'support': 1569},
  '1': {'precision': 0.8108199492814877,
   'recall': 0.9712434183880113,
   'f1-score': 0.8838109278540495,
   'support': 4938},
  'accuracy': 0.8062086983248808,
  'macro avg': {'precision': 0.7854775422083113,
   'recall': 0.629025150876606,
   'f1-score': 0.6501423912754745,
   'support': 6507},
  'weighted avg': {'precision': 0.7985985763914266,
   'recall': 0.8062086983248808,
   'f1-score': 0.7711243030217814,
   'support': 6507}},
 'request': {'0': {'precision': 0.8987854251012146,
   'recall': 0.9861188228761799,
   'f1-score': 0.9404289118347896,
   'support': 5403},
  '1': {'precision': 0.8704663212435233,
   'recall': 0.45652173913043476,
   'f1-score': 0.5989304812834224,
   'support': 1104},
  'accuracy': 0.8962655601659751,
  'macro avg': {'precision': 0.884625873172369,
   'recall': 0.7213202810033074,
   'f1-score': 0.76967969655

In [19]:
pd.DataFrame.from_dict(results_dict)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,"{'precision': 0.7601351351351351, 'recall': 0....","{'precision': 0.8987854251012146, 'recall': 0....","{'precision': 0.9960043030582449, 'recall': 1....","{'precision': 0.7767193308550185, 'recall': 0....","{'precision': 0.919432011112826, 'recall': 0.9...","{'precision': 0.9501543209876543, 'recall': 0....","{'precision': 0.9713890170742963, 'recall': 1....","{'precision': 0.9797047970479705, 'recall': 0....","{'precision': 0.9667180277349768, 'recall': 0....","{'precision': 0.9488583046606194, 'recall': 0....",...,"{'precision': 0.9889349930843707, 'recall': 1....","{'precision': 0.9568024596464259, 'recall': 0....","{'precision': 0.882387590810917, 'recall': 0.9...","{'precision': 0.9553772070626003, 'recall': 0....","{'precision': 0.9446248782072101, 'recall': 0....","{'precision': 0.9889332923455272, 'recall': 0....","{'precision': 0.9752962777499583, 'recall': 0....","{'precision': 0.9793877864943855, 'recall': 1....","{'precision': 0.95, 'recall': 0.99967621822891...","{'precision': 0.8603211776513884, 'recall': 0...."
1,"{'precision': 0.8108199492814877, 'recall': 0....","{'precision': 0.8704663212435233, 'recall': 0....","{'precision': 0.0, 'recall': 0.0, 'f1-score': ...","{'precision': 0.7957330912392192, 'recall': 0....","{'precision': 0.7142857142857143, 'recall': 0....","{'precision': 0.8888888888888888, 'recall': 0....","{'precision': 1.0, 'recall': 0.03125, 'f1-scor...","{'precision': 0.3333333333333333, 'recall': 0....","{'precision': 0.5294117647058824, 'recall': 0....","{'precision': 0.911504424778761, 'recall': 0.2...",...,"{'precision': 0.0, 'recall': 0.0, 'f1-score': ...","{'precision': 0.0, 'recall': 0.0, 'f1-score': ...","{'precision': 0.8592644978783592, 'recall': 0....","{'precision': 0.924187725631769, 'recall': 0.4...","{'precision': 0.8108882521489972, 'recall': 0....","{'precision': 0.0, 'recall': 0.0, 'f1-score': ...","{'precision': 0.8875968992248062, 'recall': 0....","{'precision': 1.0, 'recall': 0.042857142857142...","{'precision': 0.7142857142857143, 'recall': 0....","{'precision': 0.8733459357277883, 'recall': 0...."
accuracy,0.806209,0.896266,0.996004,0.783157,0.918549,0.9499,0.971415,0.979407,0.965576,0.94821,...,0.988935,0.956508,0.877363,0.954049,0.937452,0.988781,0.968342,0.979407,0.949746,0.86138
macro avg,"{'precision': 0.7854775422083113, 'recall': 0....","{'precision': 0.884625873172369, 'recall': 0.7...","{'precision': 0.49800215152912247, 'recall': 0...","{'precision': 0.7862262110471189, 'recall': 0....","{'precision': 0.8168588626992701, 'recall': 0....","{'precision': 0.9195216049382715, 'recall': 0....","{'precision': 0.9856945085371481, 'recall': 0....","{'precision': 0.656519065190652, 'recall': 0.5...","{'precision': 0.7480648962204296, 'recall': 0....","{'precision': 0.9301813647196902, 'recall': 0....",...,"{'precision': 0.49446749654218536, 'recall': 0...","{'precision': 0.4784012298232129, 'recall': 0....","{'precision': 0.8708260443446381, 'recall': 0....","{'precision': 0.9397824663471847, 'recall': 0....","{'precision': 0.8777565651781036, 'recall': 0....","{'precision': 0.4944666461727636, 'recall': 0....","{'precision': 0.9314465884873823, 'recall': 0....","{'precision': 0.9896938932471928, 'recall': 0....","{'precision': 0.8321428571428571, 'recall': 0....","{'precision': 0.8668335566895884, 'recall': 0...."
weighted avg,"{'precision': 0.7985985763914266, 'recall': 0....","{'precision': 0.893980708540758, 'recall': 0.8...","{'precision': 0.9920245717105404, 'recall': 0....","{'precision': 0.7846497666445869, 'recall': 0....","{'precision': 0.9023443681313761, 'recall': 0....","{'precision': 0.9468872078881811, 'recall': 0....","{'precision': 0.9722332323381253, 'recall': 0....","{'precision': 0.9664932702807895, 'recall': 0....","{'precision':

In [21]:
weighted_avg = {}
for key in results_dict.keys():
    weighted_avg[key] = results_dict[key]['weighted avg']

df_wavg = pd.DataFrame(weighted_avg).transpose()
df_wavg

,precision,recall,f1-score,support
related,0.798599,0.806209,0.771124,6507.0
request,0.893981,0.896266,0.882489,6507.0
offer,0.992025,0.996004,0.994010,6507.0
aid_related,0.784650,0.783157,0.778731,6507.0
medical_help,0.902344,0.918549,0.883507,6507.0
medical_products,0.946887,0.949900,0.929101,6507.0
search_and_rescue,0.972233,0.971415,0.958197,6507.0
security,0.966493,0.979407,0.969668,6507.0
military,0.951597,0.965576,0.951062,6507.0
water,0.946390,0.948210,0.933750,6507.0


In [22]:
df_wavg['f1-score'].describe()

count    35.000000
mean      0.931140
std       0.057876
min       0.771124
25%       0.916096
50%       0.944198
75%       0.971679
max       0.994010
Name: f1-score, dtype: float64

In [23]:
df_wavg[df_wavg['f1-score'] <= df_wavg['f1-score'].quantile(0.25)]

,precision,recall,f1-score,support
related,0.798599,0.806209,0.771124,6507.0
request,0.893981,0.896266,0.882489,6507.0
aid_related,0.784650,0.783157,0.778731,6507.0
medical_help,0.902344,0.918549,0.883507,6507.0
food,0.922070,0.926387,0.914761,6507.0
other_aid,0.845730,0.873060,0.817647,6507.0
infrastructure_related,0.875334,0.935147,0.904252,6507.0
weather_related,0.875941,0.877363,0.872271,6507.0
direct_report,0.862917,0.861380,0.836987,6507.0


In [25]:
df_wavg.sort_values('f1-score').head(n = 10)

,precision,recall,f1-score,support
related,0.798599,0.806209,0.771124,6507.0
aid_related,0.784650,0.783157,0.778731,6507.0
other_aid,0.845730,0.873060,0.817647,6507.0
direct_report,0.862917,0.861380,0.836987,6507.0
weather_related,0.875941,0.877363,0.872271,6507.0
request,0.893981,0.896266,0.882489,6507.0
medical_help,0.902344,0.918549,0.883507,6507.0
infrastructure_related,0.875334,0.935147,0.904252,6507.0
food,0.922070,0.926387,0.914761,6507.0
shelter,0.927412,0.932073,0.917431,6507.0


In [27]:
df_wavg[df_wavg['f1-score'] >= df_wavg['f1-score'].quantile(0.75)]

,precision,recall,f1-score,support
offer,0.992025,0.996004,0.994010,6507.0
clothing,0.983620,0.985708,0.980129,6507.0
missing_people,0.989809,0.989703,0.984731,6507.0
electricity,0.979456,0.981405,0.973295,6507.0
tools,0.987743,0.993853,0.990789,6507.0
hospitals,0.978296,0.989089,0.983663,6507.0
shops,0.989271,0.994621,0.991939,6507.0
aid_centers,0.977992,0.988935,0.983433,6507.0
fire,0.977991,0.988781,0.983356,6507.0


In [ ]:
def model_performance(labels, preds):
    # Print/collect results
    results_dict = {}
    for pred, label, col in zip(preds.transpose(), labels.values.transpose(), labels.columns):
        print(col)
        print(classification_report(label, pred))
        results_dict[col] = classification_report(label, pred, output_dict=True)
    
    # Convert to df
    weighted_avg = {}
    for key in results_dict.keys():
        weighted_avg[key] = results_dict[key]['weighted avg']

    df_wavg = pd.DataFrame(weighted_avg).transpose()
    
    # Show metrics for "key categories"
    key_categories = ['related', 'request', 'aid_related', 'medical_help',
                      'shelter', 'other_aid', 'infrastructure_related',
                      'weather_related', 'direct_report']
    
    print(df_wavg.loc[key_categories]['f1-score'].describe())
    
    return df_wavg

---


In [38]:
df_feats = df.iloc[:,4:]

In [39]:
df_feats_counts = df_feats.sum()

In [42]:
(df_feats_counts/df_feats.shape[0]).sort_values()

offer                     0.004534
shops                     0.004610
tools                     0.006109
fire                      0.010834
hospitals                 0.010873
missing_people            0.011449
aid_centers               0.011872
clothing                  0.015560
security                  0.018096
cold                      0.020363
electricity               0.020440
money                     0.023206
search_and_rescue         0.027816
military                  0.033041
refugees                  0.033618
other_infrastructure      0.044222
death                     0.045874
transport                 0.046143
medical_products          0.050446
buildings                 0.051214
other_weather             0.052866
water                     0.064239
infrastructure_related    0.065506
medical_help              0.080068
floods                    0.082795
shelter                   0.088904
storm                     0.093860
earthquake                0.094321
food                

In [43]:
df_wavg['f1-score'].sort_values()

related                   0.771124
aid_related               0.778731
other_aid                 0.817647
direct_report             0.836987
weather_related           0.872271
request                   0.882489
medical_help              0.883507
infrastructure_related    0.904252
food                      0.914761
shelter                   0.917431
other_weather             0.926304
buildings                 0.926983
medical_products          0.929101
storm                     0.929327
water                     0.933750
other_infrastructure      0.935546
transport                 0.940107
death                     0.944198
floods                    0.947253
military                  0.951062
refugees                  0.952750
search_and_rescue         0.958197
money                     0.963854
earthquake                0.967192
security                  0.969668
cold                      0.970064
electricity               0.973295
clothing                  0.980129
fire                

In [36]:
df.shape[0]

26028

In [ ]:
df_feats.sort_values()

---

### 6. Improve your model
Use grid search to find better parameters. 

In [44]:
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000022E4B4A5CA8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [ ]:
parameters = {'vect__ngram_range':[(1,2),(2,2)],
            'clf__estimator__n_estimators':[50, 100]
             }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters)

In [46]:
%%time
#grid search

parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25, 50, 100], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

model_cv = GridSearchCV(estimator=model, param_grid=parameters, verbose=3)

Wall time: 0 ns


In [49]:
%%time
model_cv.fit(X_train, Y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.218, total=  35.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.3s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.225, total=  36.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.225, total=  36.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.239, total=  35.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.235, total=  38.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.226, total=  34.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__mi

[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 379.4min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [50]:
model_cv.best_params_

{'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 100,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

In [51]:
model_cv.best_score_

0.2567491761298042

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.